# Template bank
**Authors:** Javier Roulet, Liang Dai, Tejaswi Venumadhav, Barak Zackay, Matias Zaldarriaga

This script illustrates how to load the template bank presented in our paper.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

Choose some bank and subbank id, load:

In [ ]:
bank_id = 'BBH_2'
i_subbank = 0

In [ ]:
freqs = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/fs_basis.npy'))
weights = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/inner_product_weights.npy'))
weighted_basis = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/svd_phase_basis.npy'))
asds = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/asds.npy'))
avg_amp, avg_phase = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/average_waveform_properties.npy')).T
components = np.load(os.path.join(f'{bank_id}/subbank_{i_subbank}/components.npy'))

basis = weighted_basis / weights  # psi_alpha(f)

The `weights` are defined by $w_k = A(f_k) \sqrt{\Delta f_k / S_n(f_k)}$.

`weighted_basis` is $V_{\alpha k} = w_k \psi_\alpha(f_k)$ in the paper. It is orthonormal:

In [ ]:
np.round(np.dot(weighted_basis, weighted_basis.T), decimals=10)

The first few basis functions $\psi_\alpha(f)$:

In [ ]:
plt.figure()
for alpha in range(4):
    plt.plot(freqs, basis[alpha])
plt.xlabel('$f$ (Hz)')
plt.ylabel(r'$\psi_\alpha$');

The phase components of the templates are placed on a grid:

In [ ]:
i, j = 0, 1
plt.figure()
plt.scatter(components[:,i], components[:,j], s=.5)
plt.gca().set_aspect('equal')
plt.xlabel(f'Axis {i}')
plt.ylabel(f'Axis {j}');

Individual templates in the frequency domain can be generated from the components as 
$$h(f; {\bf c}) = \overline A(f) \exp \left[i\left(\overline \psi(f) + \sum_\alpha c_\alpha \psi_\alpha(f) \right)\right]$$

In [ ]:
def gen_waveforms_fd_from_components(
        avg_amp, avg_phase, components, basis,
        fs_in=None, fs_out=None):
    """
    Generates waveforms from given coefficients
    Note: At fs_out <= min(fs_in) or >= max(fs_in), the output
    amplitudes are zero
    :param avg_amp: len(n_freqs) array of reference amplitude A(f)
                    used for building the bank.
    :param avg_phase: len(n_freqs array of average phase profile
                      used for building the bank.
    :param components: n_wf x n_(basis elements) array with list of
                       coefficients (can be vector for n_wf = 1)
    :param basis: n_basis x n_freqs array of basis phase functions.
    :param fs_in: Array of input frequencies.
    :param fs_out: Array of output frequencies. None indicates
                   fs_out = fs_in
    :return: n_wf x len(fs_out) complex array with waveforms at fs_out
    """
    phase = avg_phase + (components[..., np.newaxis] * basis).sum(axis=-2)
    if fs_out is not None:  # Upsample
        if len(phase.shape) == 1:
            phase = np.interp(fs_out, fs_in, phase)
        else:
            phase = np.array([np.interp(fs_out, fs_in, x) for x in phase])
        avg_amp = np.interp(fs_out, fs_in, avg_amp, left=0, right=0)
    waveform = avg_amp * np.exp(1j * phase)
    return waveform

In [ ]:
i_template = 0  # Choose one template in the bank
f_highres = np.linspace(0, 512, 2**20+1)
h = gen_waveforms_fd_from_components(avg_amp, avg_phase, components[i_template],
                                     basis, fs_in=freqs, fs_out=f_highres)

In [ ]:
plt.figure()
plt.plot(f_highres, h.real)
plt.xlabel('$f$ (Hz)')
plt.ylabel(r'Re $\tilde h$');